In [140]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms

class MLP(nn.Module):
  def __init__(self,input,output):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(input, 16),
      nn.ReLU(),
      nn.Linear(16,20),
      nn.ReLU(),
      nn.Linear(20,output),
      nn.Softmax(dim=1)
    )
  def forward(self,x):
    return self.layers(x)

In [141]:
import torch

class DataSet(torch.utils.data.Dataset):
    def __init__(self,data,labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        return self.data[idx], self.labels[idx]

In [142]:
import numpy as np
from sklearn.model_selection import train_test_split

data = np.genfromtxt('iris.data',delimiter=',')
data = np.delete(data,-1,axis=1)
classes = np.genfromtxt('iris.data',delimiter=',', dtype=str,usecols=[-1])
classes = np.unique(classes,return_inverse=1)[1]

data_treino, data_teste, classe_treino, classe_teste = train_test_split(data,classes,test_size=0.2,stratify=classes,random_state=1)

dataset = DataSet(data_treino,classe_treino)
data_train = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

dataset = DataSet(data_teste,classe_teste)
data_test = DataLoader(dataset, batch_size=len(data_teste), shuffle=True, num_workers=0)

In [143]:
mlp = MLP(4,3)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [144]:
epochs = 1000

mlp.train(True)
for epoch in range(epochs):
    print(f'Starting epoch {epoch+1}')

    current_loss = 0.0
    total_samples = 0

    for data in data_train:
        x,y = data
        x = x.to(torch.float32)
        optimizer.zero_grad()
        output = mlp(x)
        loss = loss_function(output,y)
        loss.backward()
        optimizer.step()
        current_loss += loss.item()
        total_samples += len(x)
    
    print(f'Loss: {current_loss/total_samples}')

print('Training process has finished.')

Starting epoch 1
Loss: 0.03746494253476461
Starting epoch 2
Loss: 0.0374871015548706
Starting epoch 3
Loss: 0.037343556682268776
Starting epoch 4
Loss: 0.03721260329087575
Starting epoch 5
Loss: 0.03734345138072968
Starting epoch 6
Loss: 0.03723272681236267
Starting epoch 7
Loss: 0.037288353840510054
Starting epoch 8
Loss: 0.03733062446117401
Starting epoch 9
Loss: 0.037313051025072735
Starting epoch 10
Loss: 0.03707557419935862
Starting epoch 11
Loss: 0.03718773523966471
Starting epoch 12
Loss: 0.03706126312414805
Starting epoch 13
Loss: 0.03695124089717865
Starting epoch 14
Loss: 0.0372205783923467
Starting epoch 15
Loss: 0.03707387149333954
Starting epoch 16
Loss: 0.03709318935871124
Starting epoch 17
Loss: 0.0369913508494695
Starting epoch 18
Loss: 0.03691519598166148
Starting epoch 19
Loss: 0.037009679277737934
Starting epoch 20
Loss: 0.03696130414803823
Starting epoch 21
Loss: 0.036997974912325544
Starting epoch 22
Loss: 0.03694234689076741
Starting epoch 23
Loss: 0.0368710984786

In [147]:
mlp.train(False)

for data in data_test:
    x_test,y_test = data
    x_test = x_test.to(torch.float32)
    y_pred = mlp(x_test)
    y_pred = torch.argmax(y_pred, dim=1)
    print(y_pred)
    print(y_test)

tensor([1, 1, 1, 2, 1, 0, 0, 2, 1, 1, 0, 2, 0, 0, 2, 0, 2, 2, 0, 1, 0, 2, 2, 0,
        2, 1, 1, 0, 1, 2])
tensor([1, 1, 1, 2, 1, 0, 0, 2, 1, 1, 0, 2, 0, 0, 2, 0, 2, 2, 0, 1, 0, 2, 2, 0,
        2, 1, 1, 0, 1, 2])
